In [1]:
##### derived from https://github.com/bozhu/AES-Python
import copy

Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text): ##
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[i // 4].append(byte)
            
    #print("{:32x}".format(text))
    #print([[hex(a) for a in m] for m in matrix])
    
    """
        A B C D E F G H I J K L M N O P

        A B C D
        E F G H
        I J K L
        M N O P
    """
    
    return matrix


def matrix2text(matrix): ##
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text


class AES:
    def __init__(self, master_key, iv=None, aes256=True):
        self.num_rounds = 14 if aes256 else 10
        self.change_key(master_key)
        self.iv = iv
    
    def matrix_xor_elementwise(self, s, k): ##
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]
                
                
    # shifts / movements only
    def matrix_shift_rows(self, s): ##
        s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

    def matrix_unshift_rows(self, s): ##
        s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]
        
    def matrix_sbox_lookup(self, s): ##
        for i in range(4):
            for j in range(4):
                s[i][j] = Sbox[s[i][j]]
                
    def matrix_invsbox_lookup(self, s): ##
        for i in range(4):
            for j in range(4):
                s[i][j] = InvSbox[s[i][j]]
                
                
    def mix_columns(self, s):
        xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)
        for i in range(4):
            t = s[i][0] ^ s[i][1] ^ s[i][2] ^ s[i][3]
            u = s[i][0]
            s[i][0] ^= t ^ xtime(s[i][0] ^ s[i][1])
            s[i][1] ^= t ^ xtime(s[i][1] ^ s[i][2])
            s[i][2] ^= t ^ xtime(s[i][2] ^ s[i][3])
            s[i][3] ^= t ^ xtime(s[i][3] ^ u)


    def unmix_columns(self, s):
        xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        self.mix_columns(s)
        
    def encrypt(self, plaintext):
        if self.iv is not None:
            self.plain_state = text2matrix(plaintext ^ self.iv)
        else:
            self.plain_state = text2matrix(plaintext)

        self.matrix_xor_elementwise(self.plain_state, self.round_keys[0])
            
        #print([hex(x) for x in self.plain_state[0]])

        for i in range(1, self.num_rounds+1):
            ## CYCLE 1
            self.matrix_sbox_lookup(self.plain_state)
            self.matrix_shift_rows(self.plain_state)
            
            #print([hex(x) for x in self.plain_state[0]])
            
            ## CYCLE 2
            if i != self.num_rounds: self.mix_columns(self.plain_state)
            self.matrix_xor_elementwise(self.plain_state, self.round_keys[i])
            
            #print([hex(x) for x in self.plain_state[0]])
            
        if self.iv is not None:
            self.iv = matrix2text(self.plain_state)
        
        return matrix2text(self.plain_state)

    def decrypt(self, ciphertext):
        self.cipher_state = text2matrix(ciphertext)

        #print(hex(self.cipher_state[3][3]))
        
        for i in range(self.num_rounds, 0, -1):
            ## CYCLE 1
            self.matrix_xor_elementwise(self.cipher_state, self.round_keys[i])
            if i != self.num_rounds: self.unmix_columns(self.cipher_state)
                
            #print(hex(self.cipher_state[3][3]))
                
            ## CYCLE 2
            self.matrix_unshift_rows(self.cipher_state)
            self.matrix_invsbox_lookup(self.cipher_state)
            
            #print(hex(self.cipher_state[0][3]))
  
        self.matrix_xor_elementwise(self.cipher_state, self.round_keys[0])
        
        out = matrix2text(self.cipher_state)
        
        if self.iv is not None:
            out = out ^ self.iv
            self.iv = ciphertext

        return out


    def change_key(self, master_key):
        
        if (self.num_rounds == 14):
            self.round_keys = [text2matrix(master_key >> 128), text2matrix(master_key & ((1 << 128) - 1))]
        else:
            self.round_keys = [text2matrix(master_key)]
        
        last_key2 = self.round_keys[0]
        last_key = self.round_keys[1] if (self.num_rounds == 14) else self.round_keys[0]

        #print([hex(x) for x in last_key[0]])
            
        for i in range(1, self.num_rounds - len(self.round_keys) + 2):
            key = []

            aes256_alt = (i % 2 == 0) and (self.num_rounds == 14)
            
            # row 0
            s0 = Sbox[last_key[3][0]]
            s1 = Sbox[last_key[3][1]]
            s2 = Sbox[last_key[3][2]]
            s3 = Sbox[last_key[3][3]]
            
            last2 = last_key2 if (self.num_rounds == 14) else last_key
            
            round_const = Rcon[i // 2 + 1] if (self.num_rounds == 14) else Rcon[i]

            r0b0 = last2[0][0] ^ (s1 if not aes256_alt else s0) ^ (round_const if not aes256_alt else 0)
            r0b1 = last2[0][1] ^ (s2 if not aes256_alt else s1)
            r0b2 = last2[0][2] ^ (s3 if not aes256_alt else s2)
            r0b3 = last2[0][3] ^ (s0 if not aes256_alt else s3)
            key.append([r0b0, r0b1, r0b2, r0b3])

            # row 1
            r1b0 = last2[1][0] ^ r0b0
            r1b1 = last2[1][1] ^ r0b1
            r1b2 = last2[1][2] ^ r0b2
            r1b3 = last2[1][3] ^ r0b3
            key.append([r1b0, r1b1, r1b2, r1b3])

            # row 2
            r2b0 = last2[2][0] ^ r1b0
            r2b1 = last2[2][1] ^ r1b1
            r2b2 = last2[2][2] ^ r1b2
            r2b3 = last2[2][3] ^ r1b3
            key.append([r2b0, r2b1, r2b2, r2b3])

            # row 3
            r3b0 = last2[3][0] ^ r2b0
            r3b1 = last2[3][1] ^ r2b1
            r3b2 = last2[3][2] ^ r2b2
            r3b3 = last2[3][3] ^ r2b3
            key.append([r3b0, r3b1, r3b2, r3b3])
          
        
            self.round_keys.append(key)
            last_key2 = last_key
            last_key = key

            #print([hex(x) for x in key[0]])
        
# 2d1541c695f88a16f8bfb5dbe3a95022

def packtext(s):
    s = s
    o = 0
    while len(s) > 0:
        o = (o << 8) | ord(s[0])
        s = s[1:]
    return o

def unpacktext(s):
    o = ""
    while s > 0:
        o = chr(s & 0xFF) + o
        s = s >> 8
    return o


key = "abcd1234ABCD!@#$zyxwZYXW*1*2*3*4"
assert len(key) == 32
iv =  "54123892jsdkjsdj"
assert len(iv) == 16


In [4]:
string1 = "helloworld123456"
string2 = "test string 1234"

print(key)

abcd1234ABCD!@#$zyxwZYXW*1*2*3*4


In [8]:
#print(ek(key.encode("ascii")))

a = AES(packtext(key), aes256=True)

enc = a.encrypt(packtext(string1))
dec = unpacktext(a.decrypt(enc))

print(packtext(key) >> 128, packtext(key) & ((1 << 128) - 1))
print(packtext(string1))
print(enc)
print(a.decrypt(enc))
print(unpacktext(a.decrypt(enc)))

print(hex(enc), dec)


assert enc == 0x2d1541c695f88a16f8bfb5dbe3a95022

##

#print(ek(key.encode("ascii")))

a = AES(packtext(key[:16]), aes256=False)

enc = a.encrypt(packtext(string1))
dec = unpacktext(a.decrypt(enc))

print(hex(enc), dec)

assert enc == 0x1708271a0a18bb2e15bd658805297b8d

129445976579865719297921356551604413220 162796526750907890493247941453607545396
138766332635719238849554048983485396278
59925632134564593020758952134513872930
138766332635719238849554048983485396278
helloworld123456
0x2d1541c695f88a16f8bfb5dbe3a95022 helloworld123456
0x1708271a0a18bb2e15bd658805297b8d helloworld123456


In [193]:
(packtext(string1) >> 128),(packtext(string1) & ((1 << 128) - 1))

(0, 138766332635719238849554048983485396278)

In [200]:


a = AES(packtext(key))

e1 = hex(a.encrypt(packtext(string1)))
#assert e1 == "0x1708271a0a18bb2e15bd658805297b8d"
e2 = hex(a.encrypt(packtext(string2)))
#assert e2 == "0x482ac205196a804865262a0044915738"
print(e1)
print(e2)

print(packtext(key), packtext(string1), int(e1, 0))

a = AES(packtext(key))
print(unpacktext(a.decrypt(int(e1, 0))))
#assert(unpacktext(a.decrypt(int(e1, 0))) == string1)
print(unpacktext(a.decrypt(int(e2, 0))))
#assert(unpacktext(a.decrypt(int(e2, 0))) == string2)



0x2d1541c695f88a16f8bfb5dbe3a95022
0xaad2f0d199eba53714f739b449c90ca6
44048183298989073697966048947243823604726133612511607082607857744689369393716 138766332635719238849554048983485396278 59925632134564593020758952134513872930
helloworld123456
test string 1234


In [201]:
hex(30614575354952859734368363414031006605)

'0x1708271a0a18bb2e15bd658805297b8d'

In [202]:
a = AES(packtext(key), packtext(iv))

e1 = hex(a.encrypt(packtext(string1)))
#assert e1 == "0x6cbaa5d41d87fc1cb2cde5f49c592554"
e2 = hex(a.encrypt(packtext(string2)))
#assert e2 == "0xb2b95376972f97140a84deda840144a2"
print(e1)
print(e2)

a = AES(packtext(key), packtext(iv))
dec1 = (unpacktext(a.decrypt(int(e1, 0))))
#assert(dec1 == string1)
print(dec1)
dec2 = (unpacktext(a.decrypt(int(e2, 0))))
#assert(dec2 == string2)
print(dec2)



0x8ea7ae770057e1522436c05b3d7448b2
0x683ec16c5385a750b7ecf9297f7b2e7e
helloworld123456
test string 1234


In [206]:
from Crypto.Cipher import AES as AE

In [208]:
print(key, len(key.encode()))
cipher = AE.new(key.encode(), AE.MODE_ECB)
ciphertext = cipher.encrypt(string1 + string2)
print(ciphertext.hex()[:32])
print(ciphertext.hex()[32:])
plaintext = cipher.decrypt(ciphertext)
print(plaintext)

abcd1234ABCD!@#$zyxwZYXW*1*2*3*4 32
2d1541c695f88a16f8bfb5dbe3a95022
aad2f0d199eba53714f739b449c90ca6
b'helloworld123456test string 1234'


In [154]:
cipher = AE.new(key.encode(), AE.MODE_CBC, iv)
ciphertext = cipher.encrypt(string1+string2)
print(ciphertext.hex()[:32])
print(ciphertext.hex()[32:])

cipher = AE.new(key.encode(), AE.MODE_CBC, iv)
plaintext = cipher.decrypt(ciphertext)
print(plaintext)

8ea7ae770057e1522436c05b3d7448b2
683ec16c5385a750b7ecf9297f7b2e7e
b'helloworld123456test string 1234'


In [15]:
import random
# Generate 256-bit encrypt test-cases

for _1 in range(10):
    key = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(32)]) # AES256 key
    print("setTopKey(BigInt(\"{}\"))".format(packtext(key) >> 128))
    print("setKey(BigInt(\"{}\"))".format(packtext(key) & ((1 << 128) - 1)))
    
    for _2 in range(10):
        plaintext = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(16)])
        iv = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(16)])
        
        c1 = AES(packtext(key))
        ct1 = c1.encrypt(packtext(plaintext))
        print("runSingleEncryptTest(BigInt(\"{}\"), BigInt(\"{}\"))"
              .format(packtext(plaintext), ct1))
        
        c2 = AES(packtext(key), iv=packtext(iv))
        ct2 = c2.encrypt(packtext(plaintext))
        print("runSingleEncryptTest(BigInt(\"{}\"), BigInt(\"{}\"), iv=BigInt(\"{}\"))"
              .format(packtext(plaintext), ct2, packtext(iv)))


setTopKey(BigInt("165406623308029099331681619155339851573"))
setKey(BigInt("105658700701783706739513152549528565804"))
runSingleEncryptTest(BigInt("67096222749627335679586640564596122443"), BigInt("278013084954040053769286071749939562985"))
runSingleEncryptTest(BigInt("67096222749627335679586640564596122443"), BigInt("124392828678387920155199777039481918860"), iv=BigInt("98873892024534219918523500662347411235"))
runSingleEncryptTest(BigInt("157026096426878869629548132200093679224"), BigInt("163243484665603839724123199786131420873"))
runSingleEncryptTest(BigInt("157026096426878869629548132200093679224"), BigInt("215429033216886894208538662194755851767"), iv=BigInt("96349157545928519709772551141028812120"))
runSingleEncryptTest(BigInt("166373065151925244287094997978767311159"), BigInt("100156579591289555453087086047683728244"))
runSingleEncryptTest(BigInt("166373065151925244287094997978767311159"), BigInt("194120200536477666012091080169261089392"), iv=BigInt("1081573336583804540544983181

In [16]:
import random
# Generate 256-bit decrypt test-cases

for _1 in range(10):
    key = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(32)]) # AES256 key
    print("setTopKey(BigInt(\"{}\"))".format(packtext(key) >> 128))
    print("setKey(BigInt(\"{}\"))".format(packtext(key) & ((1 << 128) - 1)))
    
    for _2 in range(10):
        plaintext = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(16)])
        iv = "".join([chr(random.randint(0x20, 0x7E)) for _ in range(16)])
        
        c1 = AES(packtext(key))
        ct1 = c1.encrypt(packtext(plaintext))
        print("runSingleDecryptTest(BigInt(\"{}\"), BigInt(\"{}\"))"
              .format(ct1, packtext(plaintext)))
        
        c2 = AES(packtext(key), iv=packtext(iv))
        ct2 = c2.encrypt(packtext(plaintext))
        print("runSingleDecryptTest(BigInt(\"{}\"), BigInt(\"{}\"), iv=BigInt(\"{}\"))"
              .format(ct2, packtext(plaintext), packtext(iv)))


setTopKey(BigInt("153423720557180764576530833117252168527"))
setKey(BigInt("56218445522235001715694128441772617779"))
runSingleDecryptTest(BigInt("218549502634221611235968548577572577803"), BigInt("166611971413254361133535418193110267737"))
runSingleDecryptTest(BigInt("235730131888855548816664693052185057147"), BigInt("166611971413254361133535418193110267737"), iv=BigInt("94640834602863508551148299614032913509"))
runSingleDecryptTest(BigInt("27502652893792207678373774154216790380"), BigInt("143755177493324503670103715785530488146"))
runSingleDecryptTest(BigInt("162929894617595328179352778199365747800"), BigInt("143755177493324503670103715785530488146"), iv=BigInt("110945540262939000639196814610351356780"))
runSingleDecryptTest(BigInt("236986021498394127807822385897946275530"), BigInt("145240533470560153135250944230727237452"))
runSingleDecryptTest(BigInt("188549392098996541011074000171597571433"), BigInt("145240533470560153135250944230727237452"), iv=BigInt("112051557271043465121891348